In [12]:
# mission to mars web scraping project
# stephen.peters@gmail.com
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [13]:
# visit our first site
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

m_data = {}

# Visit redplanetscience.com/
url = "https://redplanetscience.com/"
browser.visit(url)

time.sleep(1)

# Scrape first page into Soup
html = browser.html
soup = bs(html, "html.parser")

title = soup.find('div', class_='content_title').text
teaser = soup.find('div', class_='article_teaser_body').text

m_data["title"] = title
m_data["teaser"] = teaser
print('-----')
print('-- show what we have scraped --')
print(f"title = {m_data['title']}")
print(f"teaser = {m_data['teaser']}")



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/sbp/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


-----
-- show what we have scraped --
title = The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
teaser = The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. 


In [14]:
# Scrape second page into Soup   
url = "https://spaceimages-mars.com/"
browser.visit(url) 

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

relative_image = soup.find('img', class_='headerimage fade-in')["src"]
featured_image_url = url + relative_image

m_data['featured_image'] = featured_image_url
print('-----')
print('-- show what we have scraped --')
print(f"image URL = {m_data['featured_image']}")

-----
-- show what we have scraped --
image URL = https://spaceimages-mars.com/image/featured/mars1.jpg


In [61]:
# Scrape third page into Soup  
url = 'https://galaxyfacts-mars.com'
tables = pd.read_html(url)
mars_info = tables[0]
mars_info.rename(columns = {0: 'Stats', 1: 'Red Planet', 2: 'Our Planet'}, inplace=True)
planets = mars_info.to_html(index=False, classes='table-striped')

m_data['planets'] = planets
print('-- show what we have scraped --')
m_data['planets']

-- show what we have scraped --


'<table border="1" class="dataframe table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Stats</th>\n      <th>Red Planet</th>\n      <th>Our Planet</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Mars - Earth Comparison</td>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

In [16]:
# Scrape fourth page into Soup  
url = "https://marshemispheres.com/"
browser.visit(url)
time.sleep(1)

html = browser.html
soup = bs(html, 'html.parser')
hemis = soup.find_all('div', class_="item")
hemisphere_image_urls = []

for hemi in hemis:
    h3 = hemi.find('h3')
    name = hemi.find('h3').text
    name_short = name.replace(' Enhanced', '')
    link = hemi.find('a')
    href = link['href']
    newUrl = 'http://marshemispheres.com/' + href
    # go to the sub page
    browser.visit(newUrl)
    time.sleep(1)
    html_sub = browser.html
    soup_sub = bs(html_sub, 'html.parser')
    big_image = soup_sub.find('img', class_='wide-image')
    img_url = url + big_image['src']
    dict = { 'title': name_short, 'img_url': img_url }
    hemisphere_image_urls.append(dict)

m_data['hemis'] = hemisphere_image_urls  
print('-- show what we have scraped --')
m_data['hemis']

-- show what we have scraped --


[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [17]:
# Quit the browser
browser.quit()